In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


# Parameters Config

In [31]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [32]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789- '

RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [44]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [45]:
ctable = CharacterTable(chars)

In [46]:
ctable.indices_char
#ctable.char_indices
#ctable.chars

{0: ' ',
 1: '-',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

# Data Generation

In [49]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b),reverse=True))  
    if key in seen:
        continue
    seen.add(key)
    a,b = sorted((a, b),reverse=True)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    #print(query)
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [50]:
print(questions[:5], expected[:5])

['3-2    ', '498-6  ', '6-4    ', '53-8   ', '878-784'] ['1   ', '492 ', '2   ', '45  ', '94  ']


# Processing

In [57]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [62]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:60000]
train_y = y[:60000]
test_x = x[60000:]
test_y = y[60000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(54000, 7, 12)
(54000, 4, 12)
Validation Data:
(6000, 7, 12)
(6000, 4, 12)
Testing Data:
(20000, 7, 12)
(20000, 4, 12)


In [63]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False  True False False False False False False
   False]
  [False False False  True False False False False False False False
   False]
  [False False False  True False False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False False  True False
   False]
  [False False  True False False False False False False False False
   False]
  [False False False False False False False False False  True False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False False  True False
   False]
  [False False  True False False False Fal

# Build Model

In [64]:
print('Build model...')

model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
    
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 128)               72192     
_________________________________________________________________
repeat_vector_11 (RepeatVect (None, 4, 128)            0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_11 (TimeDis (None, 4, 12)             1548      
_________________________________________________________________
activation_11 (Activation)   (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [65]:
for iteration in range(40):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 22s 405us/step - loss: 1.7627 - acc: 0.3512 - val_loss: 1.6310 - val_acc: 0.3882
Q 581-28  T 553  ☒ 440 
Q 796-611 T 185  ☒ 78  
Q 803-209 T 594  ☒ 20  
Q 546-193 T 353  ☒ 20  
Q 722-2   T 720  ☒ 202 
Q 935-80  T 855  ☒ 460 
Q 799-574 T 225  ☒ 88  
Q 399-49  T 350  ☒ 462 
Q 864-378 T 486  ☒ 40  
Q 685-90  T 595  ☒ 780 

--------------------------------------------------
Iteration 1
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 19s 353us/step - loss: 1.5441 - acc: 0.4250 - val_loss: 1.4522 - val_acc: 0.4689
Q 435-8   T 427  ☒ 446 
Q 772-328 T 444  ☒ 616 
Q 239-3   T 236  ☒ 272 
Q 100-72  T 28   ☒ 10  
Q 227-98  T 129  ☒ 106 
Q 902-434 T 468  ☒ 176 
Q 572-446 T 126  ☒ 41  
Q 878-708 T 170  ☒ 10  
Q 620-13  T 607  ☒ 516 
Q 160-27  T 133  ☒ 104 

-----------------

KeyboardInterrupt: 

# Testing

In [56]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 
#####################################################
equalAnswer = 0
predictions = model.predict_classes(test_x)

for i in range(len(test_x)):
    predAnswer = ctable.decode(predictions[i],False)
    correctAnswer = ctable.decode(test_y[i])
    if(predAnswer == correctAnswer):
        equalAnswer+=1        

accuracyAnswer = equalAnswer/len(test_y)
accuracyAnswer

MSG : Prediction


0.8948833333333334